In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'search-engine-project:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4780943%2F8097020%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240419%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240419T084544Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D08e190170a1cbc70fe5af4101204b40fb71a2b6964b7e5422b126d6914c7c738cffa160db601c759be34705e388435ef7bc0c42a2d8b6ddc64f0654929b18ef4140816a794c77743ee30fcc20a42dda2641a22afb39bb774cf80c07a5a7797b86d8a23e88b16a165b3e378d5d2a13eed3e5dddec5972b3a48bf37e5f3ff237f0f5ce3dd1a033c88f1072aba8cff6b82db103094b840f9805a30515cbb9b2c275aae1fd82e966346dd23ea25cead4e428997ae1e04f21887916d2bdb34e0d3a858f5d8e52a1c671b55cf190ad1a6616f224ad999f0c3b85ff650336c7cda6751ddf87d2b78029d0d38d2a516c320262cf0dbd6a0c29abc94453c41cf450a99947,eng-subtitles-database-db:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4822036%2F8152764%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240419%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240419T084544Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dacb6950c72237fb141d233af84656ec8978739adabcd864bf9e890f24e7ad0b56b3ce504be384ced650e5c1b6bd78a0544fd2d1ad8773ede1216a9c93eebe8b72efa00d08f97c459c7e4bfbdd19e205d75de730501dff350485407bdddb2a2f5c7d75a8968d336a30214552e9636dec7c4f3a908f1960968e2c314fbcf8d81c485ba8e73dedb0abb174a81c0b5ee64f2a2fa997c79bcdb4dbff5e9b4f2a8937dcf6f98cda043845a05346c2e9e27f3b03aecb7e4fe10536be3165221a274faf9847acbc6d6aada7efb18d06368e188995a38eb098cd7c8322fb3379720fa04192bdfa3ed75eacec3cc70990f80329f9d3aa2ba743295aa31b670c1a0a4531ee7,cleaned-subtitless:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4826180%2F8158090%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240419%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240419T084544Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3400b81c815a592fb1a06a127199401271738435850c878e57ca1e67469368e9a656cf69f06e0fcb55ed05bbf5103fb57e366761b4285c22ab9d94a8e190b1630db1106036159bb84f23fbc71ead049dc0822deaa212c41fe60d00098bbd00b474242c2dd2f4c839fdc324b8c23e2a8c4bed45b151aa6fa7d6b566a886c6a669267f0aabc6c5893766aeb2f37612a5b858dc3acad18fa9ac20d3db011fd40e4fdbc021cd94d8566f09b4002a8a1aaaf32780aee41164f971b0dc360791d45cee037465499d9d9952b560e58733766bc589d4589dec81b96d0faef2c35f39185172c8b9662ac7b77643026cb8e521cf66e70e8ef87d59341ff8d575a2d18b7c75'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


* num: Unique Subtitle ID reference for www.opensubtitles.org
* name: Subtitle File Name
* content: Subtitle file were compressed and stored as a binary using 'latin-1' encoding.

## **Reading the Tables from Database file**

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
# Read the code below and write your observation in the next cell

conn = sqlite3.connect(r"/kaggle/input/eng-subtitles-database-db/eng_subtitles_database.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


**In the above cell, I am able to read the table inside the database. As mentioned earlier, table name is `zipfiles`. We also know from README.txt that this table contains three columns: 'num', 'name' and 'content'.**

## **Reading the columns of Table**

In [ ]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


**The above code helps in checking the column names in the database table.**

**Let's now use `SELECT * FROM zipfiles` to read all the data into a `df` variable.**

## **Loading the Database Table inside a Pandas DataFrame**

In [ ]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [ ]:
df.shape

(82498, 3)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


**Looks like the `content` column donot contain the subtitles text. Instead as mentioned in README.txt, it might be latin-1 encoded.**

## **Printing `content` of 0th Row**

In [ ]:
b_data = df.iloc[0, 2]

# here 2 represent the index of content column
# 0 represents the row number

In [ ]:
print(b_data)

b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x99V\x9fx\x96\xf0\x8c\x9e\x00\x00\x86\x9b\x01\x00;\x00\x00\x00The.Message.1976.REMASTERED.1080p.BluRay.x264-PiGNUS.EN.srt\xad\xbdm\x93\xdc\xc6\x91.\xfa\x9d\x11\xfc\x0f-}\xe1=\x11-\x9d\x06P\x85\x17\x9d\x8d\xd5%%[\xa4-Y>&u\x15>\xdf\xd0\xd3\x98\x19x\xfae\x0cts<\xfe\xf57\x9f\'\xb3\n\xd9\xa4\xbc\xbb\xf7\xc6Fl\xacELW\xa2\xaa\x90\x95\x95\xafO\x16/_l6\xdf\xe0\xff\xea\xf5f\xb3Y}\xf5\xd5\xbf\xaf\xf4AQ\xae7Mx\xf9\xe2\xd7\xfe|s\xbf\xea\x8f\xcf\xab\x8f\xe3n8\xadN\xc7\xfdx\x1cVO\xe3\xf9~\xf5\xf3\xe3p\xfc\xea\xfd/o>\xbc\xfb\xf0\xe3\xef\xde\xbf|\xf1\xfbi\x18Vo\xa6\xd3\xd3<L\xab\xe1\x1f\xe7\xe18\x8f\xa7\xe37\xab\xd3\xbc\xdb~-\xc3\x1e\xfe\xa7<|\xf9\xe2\xe5\x8bR_[~S\xd6\xeb\xa2k\xf3k\xe5A\xb7\xeeb\xf5\xf2\xc5\xbb\xe3\xea|?\xac\x8e\xfdaX\x9dnW?\x9cvk>8\x9c\xe6\xf3\xean\xeao\xc6\xd3ev\x8f~\x1a\xa6\x9b\xf1\xf6\xb2\xff\x1a\xe4\xabD\xbe*d\x11\xa5#_U\xeb\xaa\xd9`\xa6\xa7\xc3\xea\xa7\xcb}\x7f8\xf4F\xf9\xa7a\x9e\x87\xe3\x9d\xcc\\\xdf\x07B!\x13\xaa\xd61n<!\xd9\xaf\xd0\

**From the content, it appears to start with the bytes "PK\x03\......", which suggests that it might be a ZIP archive file. How do I know it? Experience! I have worked with something similar earlier.**

## **Unzipping the content of 385th row and decoding using `latin-1`**

In [ ]:
import zipfile
import io

# Assuming 'content' is the binary data from your database
binary_data = df.iloc[385, 2]

# Decompress the binary data using the zipfile module
with io.BytesIO(binary_data) as f:
    with zipfile.ZipFile(f, 'r') as zip_file:
        # Reading only one file in the ZIP archive
        subtitle_content = zip_file.read(zip_file.namelist()[0])

# Now 'subtitle_content' should contain the extracted subtitle content
print(subtitle_content.decode('latin-1'))  # Assuming the content is latin-1 encoded text

1
00:00:06,000 --> 00:00:12,074
Watch any video online with Open-SUBTITLES
Free Browser extension: osdb.link/ext

2
00:00:15,370 --> 00:00:16,506
You lose everything, my girl.

3
00:00:16,530 --> 00:00:19,360
So you've said - four times.

4
00:00:20,330 --> 00:00:22,120
I definitely had
it on yesterday.

5
00:00:22,465 --> 00:00:25,785
Your gloves, your keys, that
handkerchief I embroidered for you

6
00:00:25,809 --> 00:00:26,168
Everything!

7
00:00:26,192 --> 00:00:27,280
Five times.

8
00:00:31,610 --> 00:00:32,920
Miss Scarlet?
- Yes.

9
00:00:36,390 --> 00:00:37,390
I'm Miss Scarlet.

10
00:00:37,872 --> 00:00:40,880
May I inquire if
you've lost something?

11
00:00:41,350 --> 00:00:42,530
Some jewellery perhaps?

12
00:00:42,870 --> 00:00:45,130
Yes, my mother's wedding ring.

13
00:00:45,220 --> 00:00:45,840
Have you found it?

14
00:00:45,950 --> 00:00:47,656
Does your ring have
an inscription?

15
00:00:48,650 --> 00:00:51,720
From my father to my mother 'For
my beloved, Livi

**Look's like it worked.**

## **Applying the above Function on the Entire Data**

In [ ]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [ ]:
df['file_content'] = df['content'].apply(decode_method)

df.head()

,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           82498 non-null  int64 
 1   name          82498 non-null  object
 2   content       82498 non-null  object
 3   file_content  82498 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [ ]:
df.tail()

,num,name,content,file_content
82493,9521935,the.prophets.game.(2000).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb8\xa6\x...,"ï»¿1\r\n00:01:16,284 --> 00:01:19,537\r\nGod,\..."
82494,9521937,west.beirut.(1998).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13\x97\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open..."
82495,9521938,frankenstein.the.true.story.(1973).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00$\x97\x9aV...,"1\r\n00:00:01,001 --> 00:00:04,630\r\n(Dramati..."
82496,9521940,frankenstein.the.true.story.(1973).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x97\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nAdvertis..."
82497,9521941,zombie.island.massacre.(1984).eng.1cd,"b'PK\x03\x04\x14\x00\x00\x00\x08\x00,\x97\x9aV...","1\r\n00:00:01,919 --> 00:00:03,253\r\n(Sharp w..."


In [ ]:
df.shape

(82498, 4)

In [ ]:
import pandas as pd
import regex as re

In [ ]:
df

,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
...,...,...,...,...
82493,9521935,the.prophets.game.(2000).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xb8\xa6\x...,"ï»¿1\r\n00:01:16,284 --> 00:01:19,537\r\nGod,\..."
82494,9521937,west.beirut.(1998).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x13\x97\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\napi.Open..."
82495,9521938,frankenstein.the.true.story.(1973).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00$\x97\x9aV...,"1\r\n00:00:01,001 --> 00:00:04,630\r\n(Dramati..."
82496,9521940,frankenstein.the.true.story.(1973).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x00\x97\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nAdvertis..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           82498 non-null  int64 
 1   name          82498 non-null  object
 2   content       82498 non-null  object
 3   file_content  82498 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [ ]:
df.columns

Index(['num', 'name', 'content', 'file_content'], dtype='object')

In [ ]:
df.drop(columns = 'content',inplace =True)

In [ ]:
df.head()

,num,name,file_content
0,9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [ ]:
df = df[0:24749]

In [ ]:
d = df['file_content'][0]

In [ ]:
re.sub('[0-9:0-9-->0-9\r\n\W ]','',d)

'...'

In [ ]:
d

'1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch any video online with Open-SUBTITLES\r\nFree Browser extension: osdb.link/ext\r\n\r\n2\r\n00:02:26,198 --> 00:02:29,953\r\nIn the name of God, the most gracious, the most Merciful.\r\n\r\n3\r\n00:02:31,072 --> 00:02:33,370\r\nFrom Muhammad, the Messenger of God\r\n\r\n4\r\n00:02:33,550 --> 00:02:36,047\r\nto Heraclius, the emperor of Byzantium.\r\n\r\n5\r\n00:02:36,407 --> 00:02:39,464\r\ngreetings to him who is the\r\nfollower of righteous guidance.\r\n\r\n6\r\n00:02:39,783 --> 00:02:42,591\r\nI bid you to hear the divine call.\r\n\r\n7\r\n00:02:43,160 --> 00:02:45,817\r\nI am the messenger of God to the people;\r\n\r\n8\r\n00:02:46,337 --> 00:02:48,784\r\naccept Islam for your salvation.\r\n\r\n9\r\n00:02:52,231 --> 00:02:54,709\r\nHe speaks of a new prophet in Arabia.\r\n\r\n10\r\n00:02:55,068 --> 00:02:57,825\r\nWas it like this when John, the Baptist\r\ncame to king Herod\r\n\r\n11\r\n00:02:58,145 --> 00:03:01,272\r\nout of the desert, 

In [ ]:
df.head()

,num,Movies&WebSeries,Subtitles
0,9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [ ]:
# Rename columns
df = df.rename(columns={'name': 'Movies&WebSeries', 'file_content': 'Subtitles'})

In [ ]:
df.head()

,num,Movies&WebSeries,Subtitles
0,9180533,the.message.(1976).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [ ]:
# pattern for links
pattern_for_link = r'(https?://\S+|www\.\S+)'

In [ ]:
def preprocess_links(data):
    d = re.sub(pattern_for_link,'',data)
    return d

In [ ]:
df['Subtitles'] = df['Subtitles'].apply(preprocess_links)

In [ ]:
pattern_for_tags = r'<.*?>'

In [ ]:
def remove_tags(data):
    d = re.sub(pattern_for_tags,'',data)
    return d

In [ ]:
df['Subtitles'] = df['Subtitles'].apply(remove_tags)

In [ ]:
pattern_for_timestamp = r'[0-9:0-9-->0-9+\r\n\ ]'

In [ ]:
def remove_timestamp(data):
    d = re.sub(pattern_for_timestamp,' ',data)
    sentences = ' '.join(d.split())
    return sentences

In [ ]:
df['Subtitles'] = df['Subtitles'].apply(remove_timestamp)

In [ ]:
df

,num,Movies&WebSeries,Subtitles
0,9180533,the.message.(1976).eng.1cd,", , Watch any video online with Open SUBTITLES..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,", , Ah! There's Princess Dawn and Terry with t..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,", , Yumi's Cells , , Episode Extremely Polite ..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,", , Watch any video online with Open SUBTITLES..."
4,9180600,broker.(2022).eng.1cd,"ï»¿ , , Watch any video online with Open SUBTI..."
...,...,...,...
24744,9279492,scorpion.s04.e15.wave.goodbye.(2018).eng.1cd,"ï»¿ , , WALTER Previously on Scorpion TOBY Tha..."
24745,9279493,scorpion.s04.e16.nerd.wind.fire.(2018).eng.1cd,"ï»¿ , , WALTER Previously on Scorpion Hey, Pat..."
24746,9279494,scorpion.s04.e17.dumbster.fire.(2018).eng.1cd,"ï»¿ , , WALTER Previously on Scorpion What's t..."
24747,9279495,scorpion.s04.e18.dork.day.afternoon.(2018).eng...,"ï»¿ , , Kid, we'll run out of time We'll make ..."


In [ ]:
sentences_to_remove = [
    "Watch any video online with Open SUBTITLES Free Browser extension osdb link ext",
    "Advertise your product or brand here contact www OpenSubtitles org today",
    "Subtitles by MemoryOnSmells Support us and become VIP member nto remove all ads from",
    "Subtitles by"
]

In [ ]:
pattern_sentence = '|'.join(re.escape(sentence) for sentence in sentences_to_remove)

In [ ]:
def remove_specific_sentences(text, sentences_to_remove):
    pattern = '|'.join(re.escape(sentence) for sentence in sentences_to_remove)
    cleaned_text = re.sub(pattern_sentence, '', text)
    return cleaned_text.strip()

In [ ]:
df['Subtitles'] = df['Subtitles'].apply(lambda x : remove_specific_sentences(x,sentences_to_remove))

In [ ]:
df.head()

,num,Movies&WebSeries,Subtitles
0,9180533,the.message.(1976).eng.1cd,", , , , In the name of God, the most gracious..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,", , Ah! There's Princess Dawn and Terry with t..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,", , Yumi's Cells , , Episode Extremely Polite ..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,", , , , Yumi's Cells , , Episode Laptop , , F..."
4,9180600,broker.(2022).eng.1cd,"ï»¿ , , , , If you're going to throw it away,..."


In [ ]:
pattern = r'\.eng\.1cd|\W'

In [ ]:
def preprocess_name(data):
    cleaned_sentence = re.sub(pattern,' ',data)
    return cleaned_sentence.strip()

In [ ]:
df['Movies&WebSeries'] = df['Movies&WebSeries'].apply(preprocess_name)

In [ ]:
df.head()

,num,Movies&WebSeries,Subtitles
0,9180533,the message 1976,", , , , In the name of God, the most gracious..."
1,9180583,here comes the grump s01 e09 joltin jack in bo...,", , Ah! There's Princess Dawn and Terry with t..."
2,9180592,yumis cells s02 e13 episode 2 13 2022,", , Yumi's Cells , , Episode Extremely Polite ..."
3,9180594,yumis cells s02 e14 episode 2 14 2022,", , , , Yumi's Cells , , Episode Laptop , , F..."
4,9180600,broker 2022,"ï»¿ , , , , If you're going to throw it away,..."


In [ ]:
name_pattern = r'episode\s\d+\s\d+'

In [ ]:
def remove_name(e):
    data = re.sub(name_pattern,'',e)
    return data

In [ ]:
df['Movies&WebSeries'] = df['Movies&WebSeries'].apply(remove_name)

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
lem = WordNetLemmatizer()
stopword  = stopwords.words('english')

In [ ]:
remove_spl_char = '[^a-zA-Z0-9\s.]'

In [ ]:
def preprocess_text(data):
    data = data.lower()
    data = re.sub(remove_spl_char,'',data)
    data = re.sub(r'\s+', ' ', data)
    return data

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
def preprocess_text(data):
    data = data.lower()
    data = re.sub(remove_spl_char,'',data)
    data = word_tokenize(data)
    data = [word for word in data if word not in stopword]
    data = ' '.join(data)
    data = re.sub(r'\s+', ' ', data)
    return data

In [ ]:
df.columns

Index(['num', 'Movies&WebSeries', 'Subtitles'], dtype='object')

In [ ]:
df['Subtitles']  = df['Subtitles'].apply(preprocess_text)

In [ ]:
df['Movies&WebSeries'] = df['Movies&WebSeries'].apply(preprocess_text)

In [ ]:
df

,num,Movies&WebSeries,Subtitles
0,9180533,message 1976,name god gracious merciful muhammad messenger ...
1,9180583,comes grump s01 e09 joltin jack boxia 1969,ah theres princess dawn terry blooney looney s...
2,9180592,yumis cells s02 e13 2022,yumis cells episode extremely polite yumi yumi...
3,9180594,yumis cells s02 e14 2022,yumis cells episode laptop first place mine mi...
4,9180600,broker 2022,youre going throw away dont give birth please ...
...,...,...,...
24744,9279492,scorpion s04 e15 wave goodbye 2018,walter previously scorpion toby thats quincy b...
24745,9279493,scorpion s04 e16 nerd wind fire 2018,walter previously scorpion hey patty made cool...
24746,9279494,scorpion s04 e17 dumbster fire 2018,walter previously scorpion whats thing happy a...
24747,9279495,scorpion s04 e18 dork day afternoon 2018,kid well run time well make youll drive lead w...


In [ ]:
#df.to_csv('cleaned_subtitles',index = False)

In [ ]:
df['Subtitles'][0]

'name god gracious merciful muhammad messenger god heraclius emperor byzantium greetings follower righteous guidance bid hear divine call messenger god people accept islam salvation speaks new prophet arabia like john baptist came king herod desert crying salvation muqawqis patriarch alexandria kisra emperor persia muhammad calls call god accept islam salvation embrace islam come desert smelling camel goat tell persia kneel muhammad messenger god gave authority god sent muhammad mercy mankind scholars historians islam university al azhar cairo high islamic congress shiat lebanon makers film honour islamic tradition holds impersonation prophet offends spirituality message therefore person mohammad shown years christ died europe sunk dark ages everywhere old civilizations falling muhammad born mecca arabia mecca rich trading city ruled merchants whose wealth multiplied unique privilege housed gods every year time great fair desert priests brought idols images gods custody kaaba holy shri

In [ ]:
df['Subtitles'].isna().sum()

0

In [ ]:
df.dropna(subset=['Subtitles'], inplace=True)

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
count_vectorizer = CountVectorizer()
tf_matrix = count_vectorizer.fit_transform(df['Subtitles'])

In [ ]:
count_vectorizer

CountVectorizer()

In [ ]:
# TF-IDF transformer and transform the TF matrix

tfidf_transformer = TfidfTransformer()
tfidf_matrix = tfidf_transformer.fit_transform(tf_matrix)

In [ ]:
tfidf_transformer

TfidfTransformer()

In [ ]:
tfidf_matrix

<24749x346812 sparse matrix of type '<class 'numpy.float64'>'
	with 24422671 stored elements in Compressed Sparse Row format>

In [ ]:
# Calculating similarity using cosine similarity

query = input()
query_vector = count_vectorizer.transform([query])
query_tfidf = tfidf_transformer.transform(query_vector)

 broker


In [ ]:
# Compute cosine similarity between the query and documents

similarity_scores = cosine_similarity(query_tfidf, tfidf_matrix)

In [ ]:
# Retrieve top similar documents

top_indices = similarity_scores.argsort()[0][::-1]
top_n = 5
retrieved_documents = [df['Subtitles'][idx] for idx in top_indices[:top_n]]

# retrieved documents
print("Top", top_n, "documents similar to query:", query)
for i, doc in enumerate(retrieved_documents, 1):
    print("Document", i, ":", doc)

Top 5 documents similar to query: broker
Document 1 : saturday ill ask terrorists threaten destroy world take day mean plans spend day aj aj daughter ah children time goes fast making sacrifices excuse waiter champagne next one uh yeah remind whats sacrifice calling saturday simple pickup hour max especially please dont say agent archer backup archer saturday day friday night dont see problem obviously talked yet actually hasnt picked phone reason oh yeah reason hey archer support us become vip member remove ads yes shower necessary lana night many fluids first gross really second hurry whole special day planned aj coney island fancy tea place moist yet firm skin doesnt happen lana im step two ten hey see pumice stone fabian wants us pick negotiation specialist broker hes worked major players mafia yakuza dictators dirty cops cops something police last night whoo hoo police horse yeah theres way happened explain lance callouses anyway brokers longtime go criminal groups means intel hes

In [ ]:
# Summarizing

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def generate_summarized_documents(query, retrieved_documents):
    summarized_documents = {}
    for i, doc in enumerate(retrieved_documents, 1):
        summary = "Summary: " + doc[:] + "" if len(doc) > 150 else "Summary: " + doc
        summarized_documents["Document " + str(i)] = summary
    return summarized_documents

In [ ]:
query = input("Enter your query: ")
query_vector = count_vectorizer.transform([query])
query_tfidf = tfidf_transformer.transform(query_vector)

# cosine similarity between the query and documents
similarity_scores = cosine_similarity(query_tfidf, tfidf_matrix)

Enter your query:  avatar


In [ ]:
# top similar documents
top_indices = similarity_scores.argsort()[0][::-1]
top_n = 5
retrieved_documents = [df['Subtitles'][idx] for idx in top_indices[:top_n]]

In [ ]:
# summarized documents
summarized_docs = generate_summarized_documents(query, retrieved_documents)

# Print the summarized documents
for doc, summary in summarized_docs.items():
    print(doc + ":")
    print(summary)
    print()

Document 1:
Summary: katara water earth fire air long ago four nations lived together harmony everything changed fire nation attacked avatar master four elements could stop world needed vanished hundred years passed brother discovered new avatar airbender named aang although airbending skills great lotto learn hes ready save anyone believe aang save world support us become vip member remove ads aang wait till see katara air temple one beautiful places world aang know youre excited hundred years since youve home thats im excited lot change time know need see wake sokka air temple come sleep temple later sokka wake theres prickle snake sleeping bag aah get get aah heh heh heh great youre awake lets go uncle want repairs made quickly possible dont want stay long risk losing trail mean avatar dont mention name docks word gets hes alive every firebender looking dont want anyone getting way getting way prince zuko captain zhao commander general iroh great hero nation retired general fire lor

In [ ]:
import joblib

# CountVectorizer
joblib.dump(count_vectorizer, 'count_vectorizer.joblib')

['count_vectorizer.joblib']

In [ ]:
# TfidfTransformer
joblib.dump(tfidf_transformer, 'tfidf_transformer.joblib')

['tfidf_transformer.joblib']

In [ ]:
joblib.dump(tfidf_matrix, 'tfidf_matrix.joblib')

['tfidf_matrix.joblib']

In [ ]:
# cosine similarity model
joblib.dump(similarity_scores, 'cosine_similarity_scores.joblib')

['cosine_similarity_scores.joblib']

In [ ]:
df

,num,Movies&WebSeries,Subtitles
0,9180533,message 1976,name god gracious merciful muhammad messenger ...
1,9180583,comes grump s01 e09 joltin jack boxia 1969,ah theres princess dawn terry blooney looney s...
2,9180592,yumis cells s02 e13 2022,yumis cells episode extremely polite yumi yumi...
3,9180594,yumis cells s02 e14 2022,yumis cells episode laptop first place mine mi...
4,9180600,broker 2022,youre going throw away dont give birth please ...
...,...,...,...
24744,9279492,scorpion s04 e15 wave goodbye 2018,walter previously scorpion toby thats quincy b...
24745,9279493,scorpion s04 e16 nerd wind fire 2018,walter previously scorpion hey patty made cool...
24746,9279494,scorpion s04 e17 dumbster fire 2018,walter previously scorpion whats thing happy a...
24747,9279495,scorpion s04 e18 dork day afternoon 2018,kid well run time well make youll drive lead w...


In [ ]:
df.to_csv('Search-Engine',index = False)

In [ ]:
df

,num,Movies&WebSeries,Subtitles
0,9180533,message 1976,name god gracious merciful muhammad messenger ...
1,9180583,comes grump s01 e09 joltin jack boxia 1969,ah theres princess dawn terry blooney looney s...
2,9180592,yumis cells s02 e13 2022,yumis cells episode extremely polite yumi yumi...
3,9180594,yumis cells s02 e14 2022,yumis cells episode laptop first place mine mi...
4,9180600,broker 2022,youre going throw away dont give birth please ...
...,...,...,...
24744,9279492,scorpion s04 e15 wave goodbye 2018,walter previously scorpion toby thats quincy b...
24745,9279493,scorpion s04 e16 nerd wind fire 2018,walter previously scorpion hey patty made cool...
24746,9279494,scorpion s04 e17 dumbster fire 2018,walter previously scorpion whats thing happy a...
24747,9279495,scorpion s04 e18 dork day afternoon 2018,kid well run time well make youll drive lead w...
